In [24]:
# !pip3 install -U openai-whisper -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install jiwer -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install librosa -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install noisereduce -i https://mirrors.aliyun.com/pypi/simple/

In [1]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from tqdm import tqdm
import torch
import numpy as np
import glob
import os

from whisper.normalizers.english import EnglishTextNormalizer
normalizer = EnglishTextNormalizer()
import jiwer

from speechbrain.inference.ASR import EncoderDecoderASR

sr = 16000

class SpeechBrainASR():
    def __init__(self):
        self.asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="/mntcephfs/data/audiow/shoinoue/Model/hf_hub/", run_opts={"device":"cuda"})
    
    def get_transcription(self, file):
        transcription = self.asr_model.transcribe_file(file)
        return transcription

torchvision is not available - cannot save figures


In [15]:
base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/"
spk = "ASI"

In [16]:
models = {
    "speechbrain": SpeechBrainASR()
}

In [17]:
files = glob.glob(base_dir + f"{spk}/transcript/*")
files.sort()
groundtruth_dir = {}
for file in files:
    f = open(file, "r")
    text = f.read()
    key = os.path.basename(file)[:-4]
    groundtruth_dir[key] = normalizer(text)

In [18]:
save = True

result = {}
result["predtext"] = {}
result["wer"] = {}
audiofiles = glob.glob(base_dir + f"{spk}/wav/*.wav")
audiofiles.sort()
for size in models:
    print(size)
    model = models[size]
    result["predtext"][size] = {}
    result["wer"][size] = {}
    for file in tqdm(audiofiles):
        key = os.path.basename(file)[:-4]
        predtext = normalizer(models[size].get_transcription(file))
        result["predtext"][size][key] = predtext
        wer = jiwer.wer(groundtruth_dir[key], [predtext])
        result["wer"][size][key] = wer
            
savefile = "./wer_speechbrain_result.npy"
if save:
    np.save(savefile, result)

speechbrain


100%|██████████| 1131/1131 [01:56<00:00,  9.69it/s]


In [8]:
import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))

In [10]:
data = np.load("./wer_speechbrain_result.npy", allow_pickle=True).item()
targets = ["speechbrain"]
basenames = np.array(list(data["predtext"][targets[0]].keys()))

wers = np.array([np.array(list(data["wer"][size].values())) for size in targets])
wers = np.mean(wers, axis=0)

sample_num = 5 

order = np.argsort(wers)
print("Lower WER")
for i, idx in enumerate(order[:sample_num]):
    basename = basenames[idx]
    wer = wers[idx]
    path = base_dir + f"{spk}/wav/{basename}.wav"
    print(basename, wer)
    play_audio(path, sr)
    
for i, idx in enumerate(order[-sample_num:]):
    basename = basenames[idx]
    wer = wers[idx]
    path = base_dir + f"{spk}/wav/{basename}.wav"
    print(basename, wer)
    play_audio(path, sr)

Lower WER
arctic_b0423 0.0


arctic_b0407 0.0


arctic_b0408 0.0


arctic_b0415 0.0


arctic_b0416 0.0


arctic_a0554 1.1666666666666667


arctic_b0328 1.25


arctic_a0276 1.4285714285714286


arctic_b0491 2.0


arctic_a0425 3.125
